In [1]:
import json, requests

In [2]:
key = "YOUR_WEATHER_CENTER_BUREAU_KEY"

In [3]:
def analyzeLocation(userMsg):
    locations = [
        u"宜蘭", 
        u"花蓮", 
        u"臺東", 
        u"台東", 
        u"澎湖", 
        u"金門", 
        u"連江", 
        u"臺北", 
        u"台北",
        u"新北", 
        u"桃園", 
        u"臺中",
        u"台中", 
        u"臺南",
        u"台南",
        u"高雄",
        u"基隆",
        u"新竹",
        u"苗栗",
        u"彰化",
        u"南投",
        u"雲林",
        u"嘉義",
        u"屏東"
    ]
    
    location = ""
    wordIndex = 0
    for county in locations:
        if userMsg.find(county) != -1:
            wordIndex = userMsg.find(county)
            
            location = userMsg[wordIndex: wordIndex + 2]
            if location[0] == u'台':
                location = u'臺' + location[1]
                
    print 'get location: ' + location
    
    # 加上縣
    if len(location) > 0:
        if (location == u"宜蘭" or location == u"花蓮" or
        location ==  u"臺東" or location == u"澎湖" or 
        location == u"金門" or location == u"連江" or
        location == u"彰化" or location == u"南投" or 
        location == u"雲林" or location == u"屏東"):
            location += u"縣"
           
    # 處理有「縣」和「市」的
    if location == u"苗栗" or location == u"嘉義" or \
    location == u"新竹":
        countyLevel = userMsg[wordIndex + 2: wordIndex + 3]
        if countyLevel == u"縣" or countyLevel == u"市":
            # 採用使用者輸入的縣或市
            location += countyLevel
        else:
            # 使用者未輸入，預設採用「市」
            location += u"市"
    
    # 加入市
    if (location == u"臺北" or location == u"新北" or
    location == u"桃園" or location == u"臺中" or 
    location == u"臺南" or location == u"高雄" or 
    location == u"基隆"):
        location += u"市"
    
    return location

In [4]:
def getWeatherResponse(cityName, key):
    rootUrl = "http://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001"
    headers = {'Authorization': key}
    requestUrl = rootUrl + '?locationName=' + cityName + '&elementName=Wx' + '&sort=time'
    r = requests.get(requestUrl, headers=headers)
    print 'weather request status code: ' + str(r.status_code)
    if r.status_code == 200:
        return r.json()
    else:
        return None

In [5]:
def getAllWeathers(result):
    weatherStates = []
    weatherElements = result['records']['location'][0]['weatherElement'][0]['time']
    for weather in weatherElements:
        startTime = weather['startTime']
        endTime = weather['endTime']
        msg = u'未來36小時預報\n' + u'從' + startTime + u'\n到' + endTime \
        + u'\n天氣情況: ' + weather['parameter']['paramterName'] + u'\n\n'
        weatherStates.append(msg)
    return weatherStates

In [6]:
def getCurrentWeather(result):
    location = result['records']['location'][0]['locationName']
    currWeather = result['records']['location'][0]['weatherElement'][0]['time'][0]
    msg = u'現在' + location + u'天氣: ' + currWeather['parameter']['paramterName']
    return msg

In [7]:
def getReplyMsg(allWeather, currWeather):
    msg = ""
    for weatherMsg in allWeathers:
        msg += weatherMsg
    return msg + currWeather

In [8]:
test = u'今天新北天氣如何？'
loc = analyzeLocation(test)

get location: 新北


In [10]:
r = getWeatherResponse(loc, key)

weather request status code: 200


In [11]:
allWeathers = getAllWeathers(r)

In [12]:
currWea = getCurrentWeather(r)

In [13]:
print getReplyMsg(allWeathers, currWea)

未來36小時預報
從2016-12-30 12:00:00
到2016-12-30 18:00:00
天氣情況: 多雲時晴

未來36小時預報
從2016-12-30 18:00:00
到2016-12-31 06:00:00
天氣情況: 晴時多雲

未來36小時預報
從2016-12-31 06:00:00
到2016-12-31 18:00:00
天氣情況: 多雲

現在新北市天氣: 多雲時晴
